<a href="https://colab.research.google.com/github/vrublevskiyvitaliy/paraphrase_identification/blob/master/Dependancy_graph_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# All installs here

# Download medium model
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
# All imports and installs should be here
import os
import numpy as np
from nltk import wordpunct_tokenize
import operator
import re, string
import math
import spacy
from nltk import Tree
# Space module import
import en_core_web_sm


In [0]:
# All constants

COLAB_ENV = "colab"
LOCAL_ENV = "local"

SENTENCE_START_TOKEN = "sentence_start"
SENTENCE_END_TOKEN = "sentence_end"
UNKNOWN_TOKEN = "unknown_token"


In [0]:
# To be able run this notebook from Google Colab and localy

def get_running_env():
  current_path = os.getcwd()
  if current_path == "/content":
    return COLAB_ENV
  return LOCAL_ENV

RUNNING_ENV = get_running_env()

In [0]:
# Supress output of the cell
%%capture
def download_corpus():
    """
      Downloading corpus files for colab research.
    """ 
    if RUNNING_ENV == LOCAL_ENV:
      return
    files = [
      'vrublevskiyvitaliy/paraphrase_identification/contents/dataset/msr-paraphrase-corpus/msr_paraphrase_train.txt',
      'vrublevskiyvitaliy/paraphrase_identification/contents/dataset/msr-paraphrase-corpus/msr_paraphrase_test.txt',
    ]
    for f in files:
       !curl --remote-name \
          -H 'Accept: application/vnd.github.v3.raw' \
          --location https://api.github.com/repos/{f}

download_corpus()

In [0]:
def get_data_location():
  return "" if RUNNING_ENV == COLAB_ENV else "./dataset/msr-paraphrase-corpus/"

def add_start_end_sentence_tokens(s):
  return "%s %s %s" % (SENTENCE_START_TOKEN, s, SENTENCE_END_TOKEN)

def load_data(_preprocess_sentence=None, _train=False, _test=False):
    "Load the MSRP dataset."
    loc = get_data_location()
    trainloc = loc + 'msr_paraphrase_train.txt'
    testloc = loc + 'msr_paraphrase_test.txt'

    if _preprocess_sentence is None:
      _preprocess_sentence = lambda x: x

    sent1_train, sent2_train, sent1_test, sent2_test = [], [], [], []
    label_train, label_dev, label_test = [], [], []

    if _train:
        with open(trainloc, 'r', encoding='utf8') as f:
            f.readline()  # skipping the header of the file
            for line in f:
                text = line.strip().split('\t')
                sent1_train.append(_preprocess_sentence(text[3]))
                sent2_train.append(_preprocess_sentence(text[4]))
                label_train.append(int(text[0]))

    if _test:
        with open(testloc, 'r', encoding='utf8') as f:
            f.readline()  # skipping the header of the file
            for line in f:
                text = line.strip().split('\t')
                sent1_test.append(_preprocess_sentence(text[3]))
                sent2_test.append(_preprocess_sentence(text[4]))
                label_test.append(int(text[0]))

    if _train and _test:
        return [sent1_train, sent2_train], [sent1_test, sent2_test], [label_train, label_test]
    elif _train:
        return [sent1_train, sent2_train], label_train
    elif _test:
        return [sent1_test, sent2_test], label_test

In [0]:
data = load_data(_preprocess_sentence=None, _train=True, _test=False)

In [0]:
def get_sample_data():
  all_data = load_data(_preprocess_sentence=None, _train=True, _test=False)
  return all_data[0][0][0], all_data[0][1][0],  all_data[1][0]

In [0]:
sample = get_sample_data()
sample

('Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.',
 'Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.',
 1)

In [89]:
nlp = en_core_web_sm.load()

def get_dependancy_graph(s, display=False):
  doc = nlp(s)
  if display:
    spacy.displacy.render(doc, style="dep", jupyter=True)
  edges = []
  nodes = [{
      "node": "ROOT",
      "token": None,
      "is_fake": True, 
  }]
  for token in doc:
    nodes.append({
        "node": "token.text",
        "token": token,
        "is_fake": False,
    })
    if token.dep_ == "ROOT":
      edges.append({
        "start": "ROOT",
        "end": token.text,
        "type": token.dep_
      })
    else:
      edges.append({
        "start": token.head.text,
        "end": token.text,
        "type": token.dep_
      })
  return {"nodes": nodes, "edges": edges}

graph1 = get_dependancy_graph(sample[0], True)
graph2 = get_dependancy_graph(sample[1], True)
print(graph1)
print(graph2)



{'nodes': [{'node': 'ROOT', 'token': None, 'is_fake': True}, {'node': 'token.text', 'token': Amrozi, 'is_fake': False}, {'node': 'token.text', 'token': accused, 'is_fake': False}, {'node': 'token.text', 'token': his, 'is_fake': False}, {'node': 'token.text', 'token': brother, 'is_fake': False}, {'node': 'token.text', 'token': ,, 'is_fake': False}, {'node': 'token.text', 'token': whom, 'is_fake': False}, {'node': 'token.text', 'token': he, 'is_fake': False}, {'node': 'token.text', 'token': called, 'is_fake': False}, {'node': 'token.text', 'token': ", 'is_fake': False}, {'node': 'token.text', 'token': the, 'is_fake': False}, {'node': 'token.text', 'token': witness, 'is_fake': False}, {'node': 'token.text', 'token': ", 'is_fake': False}, {'node': 'token.text', 'token': ,, 'is_fake': False}, {'node': 'token.text', 'token': of, 'is_fake': False}, {'node': 'token.text', 'token': deliberately, 'is_fake': False}, {'node': 'token.text', 'token': distorting, 'is_fake': False}, {'node': 'token.te

In [0]:
for node in graph1["nodes"]:
  if not node["is_fake"]:
    assert node["token"].vector.any()
